# Overview

This scraper loads the provided Box Office Mojo data file containing movie titles and gross earnings and will scrape the IMDB-affiliated Box Office Mojo site to get each entry's IMDB id and production budget. In our main notebook, we'll connect this data set to our larger IMDB-based data set on the IMDB user ID.

# Library Imports

In [57]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup as bs

# Data Import

In [76]:
#Box Office Mojo movie gross
mojo = pd.read_csv('zippedData/bom.movie_gross.csv')

In [77]:
mojo

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010
...,...,...,...,...,...
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018


### Data Cleanup

We're going to clean up this dataset in the following way before moving on:

    a) Getting rid of all entries with no information on domestic gross
    b) Get rid of bespoke productions by eliminating all entries that are a studio's only movie
    c) Drop any rows with no studio entry

We want information on both gross and studio in order to make recommendations

In [78]:
# getting rid of all entries with no information for domestic gross
mojo.drop(mojo[(mojo['domestic_gross'].isnull())].index, inplace=True)

# dropping studio counts of only 1
counts = mojo['studio'].value_counts()
mojo.drop(mojo[mojo['studio'].isin(counts[counts == 1].index)].index, inplace=True)

# dropping null studio entries
mojo.drop(mojo[mojo['studio'].isnull()].index, inplace=True)

In [79]:
# prep our dataframe for our new budget and imdb ID columns
mojo['budget'] = ''
mojo['tmdb_id'] = ''

In [80]:
# our dataframe is ready for our new budget and tmdb entries
mojo

,title,studio,domestic_gross,foreign_gross,year,budget,tmdb_id
0,Toy Story 3,BV,415000000.0,652000000,2010,,
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010,,
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010,,
3,Inception,WB,292600000.0,535700000,2010,,
4,Shrek Forever After,P/DW,238700000.0,513900000,2010,,
...,...,...,...,...,...,...,...
3381,Beauty and the Dogs,Osci.,8900.0,NaN,2018,,
3382,The Quake,Magn.,6200.0,NaN,2018,,
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018,,
3384,El Pacto,Sony,2500.0,NaN,2018,,


# Scraping Functions

In [65]:
# function that converts our string to a search string
def format_title(title):
    text = str(title)
    result = re.sub(r"[,@\'?\.$%_:â()-]", "", text, flags=re.I)
    result = re.sub(r"\s+","+", result, flags = re.I)
    return result

In [66]:
# function that runs a search using the title string
def search(title):
    search_string = format_title(title)
    searchurl = 'https://www.boxofficemojo.com/search/?q=' + search_string
    print(searchurl)
    page = requests.get(searchurl)
    soup = bs(page.content, 'html.parser')
    return soup

In [67]:
# function that finds the url appendation for the first movie return on a page
def find_entry(soup):
    movie_url = soup.find('div', class_='a-fixed-left-grid-col a-col-right').find('a').get('href')
    print(movie_url)
    return movie_url

In [68]:
# function that receives the followurl appendation for the IMDB id and follows it
def follow_imdb(movie_url):
    followurl = 'https://www.boxofficemojo.com' + movie_url
    page = requests.get(followurl)
    soup = bs(page.content, 'html.parser')
    return soup

In [69]:
# function that takes the movie url and extracts the IMDB id
def imdb_id(movie_url):
    string = movie_url.replace('/title/', "")
    string2 = re.sub("/.*$", "", string)
    return string2

In [70]:
# function that takes in the product page and gets the budget number
def movie_budget(movie_page):
    try:
        search1 = movie_page.find('div', class_='a-section a-spacing-none mojo-summary-values mojo-hidden-from-mobile')
        search2 = search1.find('div', class_='a-section a-spacing-none').nextSibling.nextSibling
        budget = search2.find('span', class_='money').text
        return budget
    except:
        return None

In [71]:
# function to convert the raw numbers into integers
def convert_numbers(x):
    '''Takes in a string formatted number that starts with $ and may include commas, and returns that 
    number as a whole integer that can be used in calculations'''
    try:
        x = x[1:]
        x = x.replace(',', '')
        x = int(x)
        return x
    except:
        return None

# Testing Set

Before we scrape for over 3k returns, we will do a small test scrape

In [83]:
temp = mojo.sort_values('domestic_gross', ascending=False).head(5)

In [84]:
temp

,title,studio,domestic_gross,foreign_gross,year,budget,tmdb_id
1872,Star Wars: The Force Awakens,BV,936700000.0,"1,131.6",2015,,
3080,Black Panther,BV,700100000.0,646900000,2018,,
3079,Avengers: Infinity War,BV,678800000.0,"1,369.5",2018,,
1873,Jurassic World,Uni.,652300000.0,"1,019.4",2015,,
727,Marvel's The Avengers,BV,623400000.0,895500000,2012,,


In [88]:
for ind in temp.index:
    try:
        title = temp['title'][ind]
        print(('Working on {}').format(title))
        soup = search(title)
        movie_url = find_entry(soup)
        imdb = imdb_id(movie_url)
        movie_page = follow_imdb(movie_url)
        budget_string = movie_budget(movie_page)
        budget = convert_numbers(budget_string)
        print(budget)
        print(imdb)
        temp['budget'][ind] = budget
        temp['tmdb_id'][ind] = imdb
    except:
        continue

Working on Star Wars: The Force Awakens
https://www.boxofficemojo.com/search/?q=Star+Wars+The+Force+Awakens
/title/tt2488496/?ref_=bo_se_r_1
245000000
tt2488496
Working on Black Panther
https://www.boxofficemojo.com/search/?q=Black+Panther


C:\Users\Wadkins\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\Wadkins\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


/title/tt1825683/?ref_=bo_se_r_1
None
tt1825683
Working on Avengers: Infinity War
https://www.boxofficemojo.com/search/?q=Avengers+Infinity+War
/title/tt4154756/?ref_=bo_se_r_1
None
tt4154756
Working on Jurassic World
https://www.boxofficemojo.com/search/?q=Jurassic+World
/title/tt0369610/?ref_=bo_se_r_1
150000000
tt0369610
Working on Marvel's The Avengers
https://www.boxofficemojo.com/search/?q=Marvels+The+Avengers
/title/tt0848228/?ref_=bo_se_r_1
220000000
tt0848228


In [86]:
temp

,title,studio,domestic_gross,foreign_gross,year,budget,tmdb_id
1872,Star Wars: The Force Awakens,BV,936700000.0,"1,131.6",2015,245000000,tt2488496
3080,Black Panther,BV,700100000.0,646900000,2018,None,tt1825683
3079,Avengers: Infinity War,BV,678800000.0,"1,369.5",2018,None,tt4154756
1873,Jurassic World,Uni.,652300000.0,"1,019.4",2015,150000000,tt0369610
727,Marvel's The Avengers,BV,623400000.0,895500000,2012,220000000,tt0848228


# The Big Scrape

We're ready to do the big scrape!

In [89]:
for ind in mojo.index:
    try:
        title = mojo['title'][ind]
        print(('Working on {}').format(title))
        soup = search(title)
        movie_url = find_entry(soup)
        imdb = imdb_id(movie_url)
        movie_page = follow_imdb(movie_url)
        budget_string = movie_budget(movie_page)
        budget = convert_numbers(budget_string)
        print(budget)
        print(imdb)
        mojo['budget'][ind] = budget
        mojo['imdb_id'][ind] = imdb
    except:
        continue

Working on Toy Story 3
https://www.boxofficemojo.com/search/?q=Toy+Story+3
/title/tt0435761/?ref_=bo_se_r_1
200000000
tt0435761
Working on Alice in Wonderland (2010)
https://www.boxofficemojo.com/search/?q=Alice+in+Wonderland+2010


C:\Users\Wadkins\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\Wadkins\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


/title/tt1014759/?ref_=bo_se_r_1
200000000
tt1014759
Working on Harry Potter and the Deathly Hallows Part 1
https://www.boxofficemojo.com/search/?q=Harry+Potter+and+the+Deathly+Hallows+Part+1
/title/tt0926084/?ref_=bo_se_r_1
None
tt0926084
Working on Inception
https://www.boxofficemojo.com/search/?q=Inception
/title/tt1375666/?ref_=bo_se_r_1
160000000
tt1375666
Working on Shrek Forever After
https://www.boxofficemojo.com/search/?q=Shrek+Forever+After
/title/tt0892791/?ref_=bo_se_r_1
165000000
tt0892791
Working on The Twilight Saga: Eclipse
https://www.boxofficemojo.com/search/?q=The+Twilight+Saga+Eclipse
/title/tt1325004/?ref_=bo_se_r_1
68000000
tt1325004
Working on Iron Man 2
https://www.boxofficemojo.com/search/?q=Iron+Man+2
/title/tt1228705/?ref_=bo_se_r_1
200000000
tt1228705
Working on Tangled
https://www.boxofficemojo.com/search/?q=Tangled
/title/tt0398286/?ref_=bo_se_r_1
260000000
tt0398286
Working on Despicable Me
https://www.boxofficemojo.com/search/?q=Despicable+Me
/title/tt13

50000000
tt1212419
Working on The Girl with the Dragon Tattoo
https://www.boxofficemojo.com/search/?q=The+Girl+with+the+Dragon+Tattoo
/title/tt1568346/?ref_=bo_se_r_1
90000000
tt1568346
Working on Love and Other Drugs
https://www.boxofficemojo.com/search/?q=Love+and+Other+Drugs
/title/tt0758752/?ref_=bo_se_r_1
30000000
tt0758752
Working on Aftershock (Tangshan Dadizhen)
https://www.boxofficemojo.com/search/?q=Aftershock+Tangshan+Dadizhen
Working on Killers
https://www.boxofficemojo.com/search/?q=Killers
/title/tt1103153/?ref_=bo_se_r_1
75000000
tt1103153
Working on Kick-Ass
https://www.boxofficemojo.com/search/?q=KickAss
/title/tt1250777/?ref_=bo_se_r_1
30000000
tt1250777
Working on Green Zone
https://www.boxofficemojo.com/search/?q=Green+Zone
/title/tt0947810/?ref_=bo_se_r_1
100000000
tt0947810
Working on Nanny McPhee Returns
https://www.boxofficemojo.com/search/?q=Nanny+McPhee+Returns
/title/tt1415283/?ref_=bo_se_r_1
35000000
tt1415283
Working on Get Him to the Greek
https://www.boxo

/title/tt1210042/?ref_=bo_se_r_1
17000000
tt1210042
Working on Furry Vengeance
https://www.boxofficemojo.com/search/?q=Furry+Vengeance
/title/tt0492389/?ref_=bo_se_r_1
35000000
tt0492389
Working on Faster
https://www.boxofficemojo.com/search/?q=Faster
/title/tt1433108/?ref_=bo_se_r_1
24000000
tt1433108
Working on The Kids Are All Right
https://www.boxofficemojo.com/search/?q=The+Kids+Are+All+Right
/title/tt0842926/?ref_=bo_se_r_1
4000000
tt0842926
Working on You Will Meet a Tall Dark Stranger
https://www.boxofficemojo.com/search/?q=You+Will+Meet+a+Tall+Dark+Stranger
/title/tt1182350/?ref_=bo_se_r_1
22000000
tt1182350
Working on The Secret in Their Eyes
https://www.boxofficemojo.com/search/?q=The+Secret+in+Their+Eyes
/title/tt1305806/?ref_=bo_se_r_1
None
tt1305806
Working on Avatar: Special Edition
https://www.boxofficemojo.com/search/?q=Avatar+Special+Edition
/title/tt0499549/?ref_=bo_se_r_1
237000000
tt0499549
Working on Leap Year
https://www.boxofficemojo.com/search/?q=Leap+Year
/tit

...

/title/tt6911608/?ref_=bo_se_r_1
75000000
tt6911608
Working on Solo: A Star Wars Story
https://www.boxofficemojo.com/search/?q=Solo+A+Star+Wars+Story
/title/tt3778644/?ref_=bo_se_r_1
275000000
tt3778644
Working on Spider-Man: Into The Spider-Verse
https://www.boxofficemojo.com/search/?q=SpiderMan+Into+The+SpiderVerse
/title/tt4633694/?ref_=bo_se_r_1
90000000
tt4633694
Working on Fifty Shades Freed
https://www.boxofficemojo.com/search/?q=Fifty+Shades+Freed
/title/tt4477536/?ref_=bo_se_r_1
55000000
tt4477536
Working on The Nun
https://www.boxofficemojo.com/search/?q=The+Nun
/title/tt5814060/?ref_=bo_se_r_1
22000000
tt5814060
Working on Monster Hunt 2
https://www.boxofficemojo.com/search/?q=Monster+Hunt+2
/title/tt6170484/?ref_=bo_se_r_1
None
tt6170484
Working on Peter Rabbit
https://www.boxofficemojo.com/search/?q=Peter+Rabbit
/title/tt5117670/?ref_=bo_se_r_1
50000000
tt5117670
Working on Mary Poppins Returns
https://www.boxofficemojo.com/search/?q=Mary+Poppins+Returns
/title/tt5028340/?

None
tt1413492
Working on Love, Simon
https://www.boxofficemojo.com/search/?q=Love+Simon
/title/tt5164432/?ref_=bo_se_r_1
17000000
tt5164432
Working on Mile 22
https://www.boxofficemojo.com/search/?q=Mile+22
/title/tt4560436/?ref_=bo_se_r_1
50000000
tt4560436
Working on Life of the Party
https://www.boxofficemojo.com/search/?q=Life+of+the+Party
/title/tt5619332/?ref_=bo_se_r_1
None
tt5619332
Working on Isle Of Dogs
https://www.boxofficemojo.com/search/?q=Isle+Of+Dogs
/title/tt5104604/?ref_=bo_se_r_1
None
tt5104604
Working on Adrift
https://www.boxofficemojo.com/search/?q=Adrift
/title/tt6306064/?ref_=bo_se_r_1
35000000
tt6306064
Working on The 15:17 to Paris
https://www.boxofficemojo.com/search/?q=The+1517+to+Paris
/title/tt6802308/?ref_=bo_se_r_1
30000000
tt6802308
Working on Early Man
https://www.boxofficemojo.com/search/?q=Early+Man
/title/tt4701724/?ref_=bo_se_r_1
None
tt4701724
Working on Peppermint
https://www.boxofficemojo.com/search/?q=Peppermint
/title/tt6850820/?ref_=bo_se_r_

/title/tt5247704/?ref_=bo_se_r_1
None
tt5247704
Working on The Bookshop
https://www.boxofficemojo.com/search/?q=The+Bookshop
/title/tt3127022/?ref_=bo_se_r_1
None
tt3127022
Working on The Old Man & the Gun
https://www.boxofficemojo.com/search/?q=The+Old+Man+&+the+Gun
/title/tt2837574/?ref_=bo_se_r_1
None
tt2837574
Working on Gringo
https://www.boxofficemojo.com/search/?q=Gringo
/title/tt3721964/?ref_=bo_se_r_1
None
tt3721964
Working on Golden Slumber
https://www.boxofficemojo.com/search/?q=Golden+Slumber
/title/tt1413529/?ref_=bo_se_r_1
None
tt1413529
Working on Kin
https://www.boxofficemojo.com/search/?q=Kin
/title/tt6017942/?ref_=bo_se_r_1
None
tt6017942
Working on The Big Bad Fox and Other Tales
https://www.boxofficemojo.com/search/?q=The+Big+Bad+Fox+and+Other+Tales
/title/tt5851904/?ref_=bo_se_r_1
None
tt5851904
Working on Traffik
https://www.boxofficemojo.com/search/?q=Traffik
/title/tt0181865/?ref_=bo_se_r_1
None
tt0181865
Working on Can You Ever Forgive Me?
https://www.boxoffice

None
tt5628302
Working on Pokemon the Movie: The Power of Us
https://www.boxofficemojo.com/search/?q=Pokemon+the+Movie+The+Power+of+Us
/title/tt8108230/?ref_=bo_se_r_1
None
tt8108230
Working on The Party (2017)
https://www.boxofficemojo.com/search/?q=The+Party+2017
/title/tt3859310/?ref_=bo_se_r_1
None
tt3859310
Working on Burning
https://www.boxofficemojo.com/search/?q=Burning
/title/tt7282468/?ref_=bo_se_r_1
None
tt7282468
Working on Foxtrot
https://www.boxofficemojo.com/search/?q=Foxtrot
/title/tt6896536/?ref_=bo_se_r_1
None
tt6896536
Working on The Sound of Music (2018 re-release)
https://www.boxofficemojo.com/search/?q=The+Sound+of+Music+2018+rerelease
Working on The Big Lebowski (20th Anniversary)
https://www.boxofficemojo.com/search/?q=The+Big+Lebowski+20th+Anniversary
Working on Manmarziyaan
https://www.boxofficemojo.com/search/?q=Manmarziyaan
/title/tt5474036/?ref_=bo_se_r_1
None
tt5474036
Working on Loveless (2018)
https://www.boxofficemojo.com/search/?q=Loveless+2018
Working

None
tt5771710
Working on What Will People Say
https://www.boxofficemojo.com/search/?q=What+Will+People+Say
/title/tt7213936/?ref_=bo_se_r_1
None
tt7213936
Working on A Ciambra
https://www.boxofficemojo.com/search/?q=A+Ciambra
/title/tt6802896/?ref_=bo_se_r_1
None
tt6802896
Working on Oolong Courtyard
https://www.boxofficemojo.com/search/?q=Oolong+Courtyard
/title/tt8549902/?ref_=bo_se_r_1
None
tt8549902
Working on Girls vs Gangsters
https://www.boxofficemojo.com/search/?q=Girls+vs+Gangsters
/title/tt7870578/?ref_=bo_se_r_1
None
tt7870578
Working on No Date, No Signature
https://www.boxofficemojo.com/search/?q=No+Date+No+Signature
/title/tt6207878/?ref_=bo_se_r_1
None
tt6207878
Working on The Apparition (2018)
https://www.boxofficemojo.com/search/?q=The+Apparition+2018
/title/tt1433822/?ref_=bo_se_r_1
17000000
tt1433822
Working on The Charmer
https://www.boxofficemojo.com/search/?q=The+Charmer
/title/tt5668850/?ref_=bo_se_r_1
None
tt5668850
Working on Elliot: The Littlest Reindeer
http

In [90]:
mojo

,title,studio,domestic_gross,foreign_gross,year,budget,tmdb_id
0,Toy Story 3,BV,415000000.0,652000000,2010,200000000,tt0435761
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010,200000000,tt1014759
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010,None,tt0926084
3,Inception,WB,292600000.0,535700000,2010,160000000,tt1375666
4,Shrek Forever After,P/DW,238700000.0,513900000,2010,165000000,tt0892791
...,...,...,...,...,...,...,...
3381,Beauty and the Dogs,Osci.,8900.0,NaN,2018,None,tt6776572
3382,The Quake,Magn.,6200.0,NaN,2018,None,tt6523720
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018,,
3384,El Pacto,Sony,2500.0,NaN,2018,None,tt0093177


In [92]:
# Setting the title as the index
mojo.set_index('tmdb_id', inplace=True)

# Data Export

In [97]:
#exporting the dataframe to a csv
mojo.to_csv('api_data/bom_imdb_boxoffice.csv')